# Facebook & Website Mining (Silver 2)

In [93]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from Apify_Scrapper_Functions import Get_Phone_Number_From_Facebook, Get_Phone_Number_From_Website

### Import scrapped data

In [94]:
Scrapped_Data = pd.read_parquet("./Staging/Silver/Silver_data_1.parquet", engine="fastparquet")

# Check if UEN is unique
print(Scrapped_Data["UEN"].is_unique)
print(Scrapped_Data.shape)

True
(10, 15)


In [95]:
Scrapped_Data.tail(50)

,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3
0,200613314G,None,None,http://atspace.sg/,None,None,None,None,1 GOLDHILL PLAZA GOLDHILL PLAZA,02-31,308899,1 GOLDHILL PLAZA GOLDHILL PLAZA 02-31 Singapor...,None,None,None
1,200708627D,[+6589318520],[info@artbuilderz.com],https://artbuilderz.com/,[https://www.facebook.com/Artbuilderz.sg],None,[https://www.instagram.com/artbuilderz/],None,22 SIN MING LANE MIDVIEW CITY,6-76,573969,22 SIN MING LANE MIDVIEW CITY 6-76 Singapore 5...,RecordOwl,None,None
2,200715323E,"[+6562277996, +6562279672]",[hr_dept@acpcomputer.edu.sg],https://acpcomputer.com/,None,[https://sg.linkedin.com/in/acp-computer-61272...,[https://www.instagram.com/acpcomputersg/],None,3 ANG MO KIO STREET 62 LINK@AMK,06-18,569139,3 ANG MO KIO STREET 62 LINK@AMK 06-18 Singapor...,RecordOwl,None,None
3,200721644W,[+6592214705],[team@controllables.sg],https://athlete.sg/,None,None,None,None,20 EASTWOOD ROAD EASTWOOD CENTRE,03-13,486442,20 EASTWOOD ROAD EASTWOOD CENTRE 03-13 Singapo...,RecordOwl,None,None
4,200806301Z,[+6565620138],[enquiries@addceledu.tech],https://addcel.com.sg/,None,None,None,None,566 HOUGANG STREET 51,06-456,530566,566 HOUGANG STREET 51 06-456 Singapore 530566,RecordOwl,None,None
5,200902560M,None,None,None,None,None,None,None,768 CHOA CHU KANG STREET 54,10-33,680768,768 CHOA CHU KANG STREET 54 10-33 Singapore 68...,None,None,None
6,200904900G,[+6563488005],[info@autismrecovery.sg],https://autismrecovery.sg/,[https://www.facebook.com/autism.singapore],[https://www.linkedin.com/company/autism-recov...,[https://www.instagram.com/arn.sg/],None,458A JOO CHIAT ROAD,None,427671,458A JOO CHIAT ROAD Singapore 427671,RecordOwl,None,None
7,200910650Z,[+6594236248],[admin@abccentersingapore.com],https://www.mycareersfuture.gov.sg/job/educati...,[https://www.facebook.com/p/Applied-Behavior-C...,[https://sg.linkedin.com/company/applied-behav...,[https://www.instagram.com/abc_center_singapore/],None,111 SOMERSET ROAD 111 SOMERSET,04-03,238164,111 SOMERSET ROAD 111 SOMERSET 04-03 Singapore...,RecordOwl,None,None
8,200912105K,[+6565153969],None,https://www.ascendo.sg/,[https://www.facebook.com/p/Ascendo-Academy-Pt...,None,None,None,395A BUKIT BATOK WEST AVENUE 5 GOODVIEW GARDENS,03-04,651395,395A BUKIT BATOK WEST AVENUE 5 GOODVIEW GARDEN...,RecordOwl,None,None
9,200923394W,None,None,None,None,None,None,None,1A JALAN AWANG CHUAN VILLAS,None,419644,1A JALAN AWANG CHUAN VILLAS Singapore 419644,None,None,None


In [96]:
print(Scrapped_Data['Phones'].dropna().loc[lambda x: x.str.strip() != ''].count())

7


### Separate W/O Phone Numbers

In [97]:
# Separate rows with and without phone numbers
df_with_phones = Scrapped_Data[Scrapped_Data["Phones"].notna()]
df_without_phones = Scrapped_Data[Scrapped_Data["Phones"].isna()]

# Store unique phone numbers inside With_Phones_1
df_With_Phones_1 = df_with_phones[df_with_phones["Phones"].duplicated(keep=False) == False]

# Store duplicate phone numbers inside refilter_df_1
df_Duplicate_Phones_1 = df_with_phones[df_with_phones["Phones"].duplicated(keep=False) == True]

# Print shapes
print("Total having Phone numbers:", df_with_phones.shape)
print("Total without Phone numbers:", df_without_phones.shape)
print("Unique phone rows (df_With_Phones_1):", df_With_Phones_1.shape)
print("Duplicate phone rows (df_Duplicate_Phones_1):", df_Duplicate_Phones_1.shape)


Total having Phone numbers: (7, 15)
Total without Phone numbers: (3, 15)
Unique phone rows (df_With_Phones_1): (7, 15)
Duplicate phone rows (df_Duplicate_Phones_1): (0, 15)


### Merge df_without_phones with df_Duplicate_Phones_1

In [98]:
df_without_phones_and_Duplicates = (
    pd.concat([df_Duplicate_Phones_1, df_without_phones], ignore_index=True)
      .drop_duplicates(subset=["UEN"])
)

df_without_phones_and_Duplicates.shape

(3, 15)

In [99]:
### Final W/O Phones 

print(df_With_Phones_1.shape)
print(df_without_phones_and_Duplicates.shape)

print(df_With_Phones_1["UEN"].is_unique)
print(df_without_phones_and_Duplicates["UEN"].is_unique)

df_without_phones_and_Duplicates["PIC Source 1"] = None

(7, 15)
(3, 15)
True
True


In [100]:
df_With_Phones_1.head(10)

,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3
1,200708627D,[+6589318520],[info@artbuilderz.com],https://artbuilderz.com/,[https://www.facebook.com/Artbuilderz.sg],None,[https://www.instagram.com/artbuilderz/],None,22 SIN MING LANE MIDVIEW CITY,6-76,573969,22 SIN MING LANE MIDVIEW CITY 6-76 Singapore 5...,RecordOwl,None,None
2,200715323E,"[+6562277996, +6562279672]",[hr_dept@acpcomputer.edu.sg],https://acpcomputer.com/,None,[https://sg.linkedin.com/in/acp-computer-61272...,[https://www.instagram.com/acpcomputersg/],None,3 ANG MO KIO STREET 62 LINK@AMK,06-18,569139,3 ANG MO KIO STREET 62 LINK@AMK 06-18 Singapor...,RecordOwl,None,None
3,200721644W,[+6592214705],[team@controllables.sg],https://athlete.sg/,None,None,None,None,20 EASTWOOD ROAD EASTWOOD CENTRE,03-13,486442,20 EASTWOOD ROAD EASTWOOD CENTRE 03-13 Singapo...,RecordOwl,None,None
4,200806301Z,[+6565620138],[enquiries@addceledu.tech],https://addcel.com.sg/,None,None,None,None,566 HOUGANG STREET 51,06-456,530566,566 HOUGANG STREET 51 06-456 Singapore 530566,RecordOwl,None,None
6,200904900G,[+6563488005],[info@autismrecovery.sg],https://autismrecovery.sg/,[https://www.facebook.com/autism.singapore],[https://www.linkedin.com/company/autism-recov...,[https://www.instagram.com/arn.sg/],None,458A JOO CHIAT ROAD,None,427671,458A JOO CHIAT ROAD Singapore 427671,RecordOwl,None,None
7,200910650Z,[+6594236248],[admin@abccentersingapore.com],https://www.mycareersfuture.gov.sg/job/educati...,[https://www.facebook.com/p/Applied-Behavior-C...,[https://sg.linkedin.com/company/applied-behav...,[https://www.instagram.com/abc_center_singapore/],None,111 SOMERSET ROAD 111 SOMERSET,04-03,238164,111 SOMERSET ROAD 111 SOMERSET 04-03 Singapore...,RecordOwl,None,None
8,200912105K,[+6565153969],None,https://www.ascendo.sg/,[https://www.facebook.com/p/Ascendo-Academy-Pt...,None,None,None,395A BUKIT BATOK WEST AVENUE 5 GOODVIEW GARDENS,03-04,651395,395A BUKIT BATOK WEST AVENUE 5 GOODVIEW GARDEN...,RecordOwl,None,None


### Separate W/O Facebook from df_without_phones_and_Duplicates

In [101]:
# DataFrame with Facebook (not null and not empty)
df_with_facebook = df_without_phones_and_Duplicates[
    df_without_phones_and_Duplicates["Facebook"].notna() &
    (df_without_phones_and_Duplicates["Facebook"] != "")
]

# DataFrame without Facebook (null or empty)
df_without_facebook = df_without_phones_and_Duplicates[
    df_without_phones_and_Duplicates["Facebook"].isna() |
    (df_without_phones_and_Duplicates["Facebook"] == "")
]

# Check the shapes
print(f"With Facebook: {df_with_facebook.shape}")
print(f"Without Facebook: {df_without_facebook.shape}")

With Facebook: (0, 15)
Without Facebook: (3, 15)


In [102]:
print(df_with_facebook["UEN"].is_unique)


True


In [103]:
df_with_facebook.shape

(0, 15)

### Extract Phone Number of Facebook Pages

In [104]:

facebook_phone_df, facebook_no_phone_df = Get_Phone_Number_From_Facebook(df_with_facebook)


Valid Facebook URLs: 0, Filtered out (profiles/invalid): 0
No valid Facebook URLs to process.
Summary: 0 with phones, 0 without phones (includes 0 filtered URLs)


In [105]:
print("with fb")
print("----------------")
print(facebook_phone_df["Phones"].apply(str).is_unique)
print(facebook_phone_df["UEN"].is_unique)
print(facebook_phone_df.shape)

print("/n without fb")
print("----------------")
print(facebook_no_phone_df["Phones"].apply(str).is_unique)
print(facebook_no_phone_df["UEN"].is_unique)
print(facebook_no_phone_df.shape)

with fb
----------------
True
True
(0, 15)
/n without fb
----------------
True
True
(0, 15)


### Adding Source to facebook_phone_df and merging with df_With_Phones_1


In [106]:
facebook_phone_df.shape

(0, 15)

### Assign PIC 1 SOurce as Facebook

In [107]:
# Assign PIC Source 1 as "RecordOwl" if Phones has any value
facebook_phone_df.loc[facebook_phone_df["Phones"].notna(), "PIC Source 1"] = "Facebook"


### 

### Merge df_With_Phones_1 with facebook_phone_df = with unique value phone number

In [108]:
df_With_Phones_2 = pd.concat([df_With_Phones_1, facebook_phone_df], ignore_index=True)


print(df_With_Phones_2["UEN"].is_unique)
print(df_With_Phones_2.shape)

True
(7, 15)


### Merging back facebook_no_phone_df with df_without_facebook = no valid number

In [109]:
df_without_phones_2 = pd.concat([df_without_facebook, facebook_no_phone_df], ignore_index=True)

print(df_without_phones_2["UEN"].is_unique)
print(df_without_phones_2.shape)

True
(3, 15)


### Quality Check df_With_Phones_2

In [110]:
# Check for NA or empty values in Phones and PIC Source 1
print("Phones - NA count:", df_With_Phones_2["Phones"].isna().sum())
print("Phones - Empty string count:", (df_With_Phones_2["Phones"] == "").sum())
print("PIC Source 1 - NA count:", df_With_Phones_2["PIC Source 1"].isna().sum())
print("PIC Source 1 - Empty string count:", (df_With_Phones_2["PIC Source 1"] == "").sum())

Phones - NA count: 0
Phones - Empty string count: 0
PIC Source 1 - NA count: 0
PIC Source 1 - Empty string count: 0


### Quality Check df_without_phones_2

In [111]:
# Check where PIC Source 1 is not NA in df_without_phones_2
print("Rows where PIC Source 1 is not NA:")
print(df_without_phones_2[df_without_phones_2["PIC Source 1"].notna()].shape)
print()
df_without_phones_2[df_without_phones_2["PIC Source 1"].notna()]

Rows where PIC Source 1 is not NA:
(0, 15)



,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3


### Separate W/O Websites

In [112]:
# DataFrame with Websites (not null and not empty)
df_with_websites = df_without_phones_2[
    df_without_phones_2["Website"].notna() &
    (df_without_phones_2["Website"] != "")
]

# DataFrame without Websites (null or empty)
df_without_websites = df_without_phones_2[
    df_without_phones_2["Website"].isna() |
    (df_without_phones_2["Website"] == "")
]

# Check the shapes
print(f"Total from df_without_phones_2: {df_without_phones_2.shape}")
print(f"With Websites: {df_with_websites.shape}")
print(f"Without Websites: {df_without_websites.shape}")


Total from df_without_phones_2: (3, 15)
With Websites: (1, 15)
Without Websites: (2, 15)


In [113]:
df_with_websites

,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3
0,200613314G,None,None,http://atspace.sg/,None,None,None,None,1 GOLDHILL PLAZA GOLDHILL PLAZA,02-31,308899,1 GOLDHILL PLAZA GOLDHILL PLAZA 02-31 Singapor...,None,None,None


### Extract Phone Numbers from Websites

In [115]:
from Apify_Scrapper_Functions import Get_Phone_Number_From_Website

website_df_with_phones, website_df_without_phones = Get_Phone_Number_From_Website(df_with_websites)

Validating Apify API token...
API Key valid - User: epos-pte-ltd
Rows with valid websites: 1
Verifying 1 websites...
  Verifying 1/1 (Verified: 0, Failed: 0, Skipped: 0)
Verified: 1, Failed: 0, Skipped: 0
Batch 1/1 - Processing 1 websites
  Starting actor: apify/puppeteer-scraper (1 URLs)


[apify.puppeteer-scraper runId:Y90DRGaFihYKoGm9Z] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:Y90DRGaFihYKoGm9Z] -> 2026-02-18T15:04:05.187Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:Y90DRGaFihYKoGm9Z] -> 2026-02-18T15:04:05.189Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:Y90DRGaFihYKoGm9Z] -> 2026-02-18T15:04:05.221Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:Y90DRGaFihYKoGm9Z] -> 2026-02-18T15:04:05.417Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:Y90DRGaFihYKoGm9Z] -> 2026-02-18T15:04:06.096Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:Y90DRGaFihYKoGm9Z] -> 2026-02-18T15:04:06.225Z INFO  Configuring Puppeteer Scraper

  Actor run started - Run ID: Y90DRGaFihYKoGm9Z | Dataset ID: vhylDggEBsy67zwHc
  Console: https://console.apify.com/actors/runs/Y90DRGaFihYKoGm9Z
  Actor run finished - Status: SUCCEEDED | Duration: 9.079s
  Results: 1 pages scraped | 1 with phone numbers
Total scraped: 1, Phones found: 1
With unique phones: 1, Without phones: 0


In [116]:
website_df_with_phones


,UEN,Phones,Emails,Website,Facebook,LinkedIn,Instagram,TikTok,operational_street,operational_unit,operational_postal_code,operational_address,PIC Source 1,PIC Source 2,PIC Source 3
0,200613314G,+6584064281,None,http://atspace.sg/,None,None,None,None,1 GOLDHILL PLAZA GOLDHILL PLAZA,02-31,308899,1 GOLDHILL PLAZA GOLDHILL PLAZA 02-31 Singapor...,Website,None,None


In [117]:
website_df_without_phones.shape

(0, 15)

### Merge df_With_Phones_2 with website_number

In [ ]:
df_With_Phones_2.shape

In [ ]:
df_With_Phones_3 = pd.concat([df_With_Phones_2, website_df_with_phones], ignore_index=True)

print(df_With_Phones_3["UEN"].is_unique)
print(df_With_Phones_3.shape)

### Merge df_without_websites with no_website_number

In [ ]:
df_without_websites.shape

In [ ]:
website_df_without_phones.shape

In [ ]:
df_without_phones_3 = pd.concat([df_without_websites, website_df_without_phones], ignore_index=True)

print(df_without_phones_3["UEN"].is_unique)
print(df_without_phones_3.shape)

### Quality Check df_With_Phones_3

In [ ]:
# Check for NA or empty values in Phones and PIC Source 1
print("Phones - NA count:", df_With_Phones_3["Phones"].isna().sum())
print("Phones - Empty string count:", (df_With_Phones_3["Phones"] == "").sum())
print("PIC Source 1 - NA count:", df_With_Phones_3["PIC Source 1"].isna().sum())
print("PIC Source 1 - Empty string count:", (df_With_Phones_3["PIC Source 1"] == "").sum())

In [ ]:
df_With_Phones_3["UEN"].is_unique

### Quality Check df_without_phones_3

In [ ]:
# Check for NA or empty values in Phones and PIC Source 1
print("Phones - NA count:", df_without_phones_3["Phones"].isna().sum())
print("Phones - Empty string count:", (df_without_phones_3["Phones"] == "").sum())
print("PIC Source 1 - NA count:", df_without_phones_3["PIC Source 1"].isna().sum())
print("PIC Source 1 - Empty string count:", (df_without_phones_3["PIC Source 1"] == "").sum())

In [ ]:
df_without_phones_3["UEN"].is_unique

In [ ]:
df_With_Phones_3

In [ ]:
df_without_phones_3

### Saving

In [ ]:
df_With_Phones_3.to_parquet("./Staging/Silver/Silver_data_2_Phone.parquet", index=False, engine="fastparquet")

In [ ]:
df_without_phones_3.to_parquet("./Staging/Silver/Silver_data_2_No_Phone.parquet", index=False, engine="fastparquet")